# Código principal

Nesse código é implementado todo o pipeline proposto pelo projeto, como definido na imagem a seguir:

![image](../diagramaPI.drawio.png)

## Instalação das bibliotecas necessárias

In [ ]:
%pip install numpy
%pip install pandas
%pip install pytesseract
%pip install opencv-python
%pip install matplotlib
%pip install imutils

## Importações de bibliotecas externas

In [ ]:
import pytesseract

Para configurar o `pytesseract` é preciso instalar o seu executável. Se não tiver configurado ele nas variáveis de ambiente, modificar a variável do código abaixo para indicar o caminho até o executável do `tesseract`.

A documentação do `pytesseract` pode ser [acessada aqui](https://pypi.org/project/pytesseract/)

O `tesseract` pode ser instalado [por aqui](https://tesseract-ocr.github.io/tessdoc/Installation.html)

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

## Importações internas

## Criando objetos da interface gráfica do projeto

## Definindo função do pipeline completo

## Mostrando interface